# DPL implementation

## Syntax

In [ ]:
data Formula = Exists Var | Even Var | DivBy3 Var | Formula `And` Formula | Not Formula

newtype Var = Var Int deriving (Eq,Ord,Show)

newtype Indiv = Indiv Int deriving (Eq,Ord)

instance Show Indiv where
  show (Indiv n) = "e" ++ show n

_even :: Indiv -> Bool
_even (Indiv n) = even n

_divBy3 :: Indiv -> Bool
_divBy3 (Indiv n) = n `rem` 3 == 0

type Assignment = [Indiv]

lookup :: Assignment -> Var -> Indiv
lookup g (Var x) = g !! x

_dom :: [Indiv]
_dom = Indiv <$> [1..10]

## Semantics

In [ ]:
insert :: [a] -> Var -> a -> [a]
insert oldList (Var index) val = if index > length oldList then undefined else concat [take index oldList, [val], drop (index + 1) oldList ]

In [ ]:
randomAssignment :: Assignment -> [Indiv] -> Var -> [Assignment]
randomAssignment g dom x = [ insert g x a | a <- dom ] 

In [ ]:
eval :: Formula -> [Indiv] -> Assignment -> [Assignment]
eval (Even x) dom g = [ g | _even (lookup g x) ]
eval (DivBy3 x) dom g = [ g | _divBy3 (lookup g x) ]
eval (Exists x) dom g = randomAssignment g dom x
eval (p `And` q) dom g = concat [ eval q dom h | h <- eval p dom g ]
eval (Not p) dom g = [ g | null $ eval p dom g ]

_Or :: Formula -> Formula -> Formula
_Or p q = Not (Not p `And` Not q)

_If :: Formula -> Formula -> Formula
_If p q = Not (p `And` Not q)

In [ ]:
randomAssignment [Indiv 3, Indiv 7] _dom (Var 0)

$g[p and q]h = \text{there is some }i g[p]i\text{ and }i[q]h$

There is some even number

$\exists x \wedge Even(x)$

In [ ]:
testExistential :: Formula
testExistential = Exists (Var 0) `And` Even (Var 0)

testNot = Not testExistential

eval testNot (Indiv <$> [3,9,5]) [Indiv 3]

1. There's no even number, and it's prime

There's an even number and it's divisible by 3.

$\exists x[even(3)] \wedge divby3(x)$

In [ ]:
eval (testExistential `And` DivBy3 (Var 0)) _dom [Indiv 3, Indiv 7]

## Negation

1. I haven't read a book by Chomsky. #It was interesting
2. Someone isn't here.
3. It's not the case that anyone is here.

$not p = \lambda g\,.\,\lbrace g | p(g) = \emptyset \rbrace$

4. There isn't an even number.

$\lambda g\,.\,\lbrace g | \text{there isn't an even number}\rbrace$

## Pretty printing the output set

`insert [Indiv 1] 0 (Indiv 2) = [Indiv 2]`

In [ ]:
f :: Assignment -> String
f g = unwords (map show g)

In [ ]:
f' :: [Assignment] -> String
f' as@(g:gs) = unlines $ unwords (intersperse "" (map show [0..(maxLength as - 1)])) : (f <$> as)

In [ ]:
ppOutput :: [Assignment] -> IO ()
ppOutput = putStrLn . f'

# Functors

In [1]:
:t (+)

(+) :: forall {a}. Num a => a -> a -> a

In [2]:
data Person = ClarkKent | Superman | Lois

instance Eq Person where
    ClarkKent == ClarkKent = True
    ClarkKent == Superman = True
    Lois == Lois = True
    _ == _ = False

In [7]:
ClarkKent == Superman

True

``` haskell
class Functor f where
    fmap :: (a -> b) -> f a -> f b
```

## Functor laws

``` haskell
fmap id = id
fmap (f . g)  ==  fmap f . fmap g
```

In [18]:
data Perhaps a = Definitely a | Oops deriving Show

In [26]:
{-# LANGUAGE InstanceSigs #-}

instance Functor Perhaps where
  fmap :: (a -> b) -> Perhaps a -> Perhaps b
  fmap f Oops = Oops 
  fmap f (Definitely x) = Definitely (f x)

In [28]:
:t even

even :: forall {a}. Integral a => a -> Bool

In [29]:
:t not

not :: Bool -> Bool

In [30]:
:t (not . even)

(not . even) :: forall {a}. Integral a => a -> Bool

In [32]:
fmap (not . even) (Definitely 3)
-- Definitely $ (not . even) 3
-- Definitely True

(fmap not . fmap even) (Definitely 3)
-- fmap not (fmap even (Definitely 3))
-- fmap not (Definitely (even 3))
-- fmap not (Definitely False)
-- Definitely (not False)
-- Definitely True

Line 1: Use odd
Found:
not . even
Why not:
oddLine 5: Functor law
Found:
fmap not . fmap even
Why not:
fmap (not . even)

Definitely True

Definitely True

In [35]:
data Counter a = Counter Int a deriving Show

In [37]:
:t (Counter 3 True)

(Counter 3 True) :: Counter Bool

In [38]:
:t (Counter 4 'a')

(Counter 4 'a') :: Counter Char

In [39]:
increment :: Counter a -> Counter a
increment (Counter n x) = Counter (n + 1) x

In [53]:
instance Functor Counter where
  fmap f (Counter n x) = Counter (n + 1) (f x)

In [56]:
fmap (not . even) $ Counter 7 3

Line 1: Use <$>
Found:
fmap (not . even) $ Counter 7 3
Why not:
(not . even) <$> Counter 7 3Line 1: Use odd
Found:
not . even
Why not:
odd

Counter 8 True

# Languages with dependent types

- agda
- idris
- coq

# Functor examples

In [59]:
fmap (+1) [1,2,3]

[2,3,4]

In [60]:
fmap (+1) $ Just 3

Line 1: Use <$>
Found:
fmap (+ 1) $ Just 3
Why not:
(+ 1) <$> Just 3

Just 4

:t (a -> b)

(->) a b

(->) a

instance Functor ((->) a) where
  fmap f (\x -> y) = \x -> f y

In [64]:
fmap odd (\x -> x + 3)
-- \x -> odd (x + 3)

(fmap odd (\x -> x + 3)) 1
-- (\x -> odd (x + 3)) 1
-- odd (1 + 3)
-- odd 4

Line 1: Avoid lambda using `infix`
Found:
(\ x -> x + 3)
Why not:
(+ 3)Line 4: Redundant bracket
Found:
(fmap odd (\ x -> x + 3)) 1
Why not:
fmap odd (\ x -> x + 3) 1Line 4: Avoid lambda using `infix`
Found:
(\ x -> x + 3)
Why not:
(+ 3)

: 

In [ ]:
[[a]] -> [a]